In [ ]:
import utils

# Number of instances to create (at least 2) 
nb_instances = 3
# Private subnet range
private_subnet = '10.0.0.0/24'
# The type of instance to use
instance_type = "t2.medium"


def create_cluster():
    """
    Sets up prerequisites and creates instances used in the project.
    When this function returns, the instances are running and ready to use.
    """
    # Get current external IP
    current_ip_address = utils.get_ext_ip()

    # Retrieve Ubuntu 20.04 LTS amd64 image id
    image_id = utils.get_image_id("/aws/service/canonical/ubuntu/server/20.04/stable/current/amd64/hvm/ebs-gp2/ami-id")

    # Create key pair
    key_pair = create_key_pair()
    print(f"Created a key pair {key_pair.key_name} and saved the private key to ~/.cbdproject/{key_pair.key_name}.pem")

    # Create VPC and subnet
    vpc, subnet = create_vpc()
    vpc.wait_until_available()
    print(f"Created a VPC with ID {vpc.id} and subnet {subnet.cidr_block}")

    # Create security group
    sec_group = create_sg(vpc)
    print(f"Created security group {sec_group.group_name} that allows SSH, HTTP and HTTPS access from {current_ip_address}.")

    vm_instances = {}

    print("Creating instances, please wait...")

    for i in range(nb_instances):
        vm_instances[f'instance_{i}'] = create_instance(image_id, f"cbdproject-inst-{i}", key_pair.key_name, subnet.id, sec_group.group_id)

    running_waiter = utils.ec2_client.get_waiter("instance_status_ok")
    running_waiter.wait(InstanceIds=[i.instance_id for i in vm_instances.values()])
    
    for i in vm_instances.values():
        i.load()
        print(f"Instance {i.instance_id} created !")
        print(f"\tAvailable at {i.public_dns_name} ({i.public_ip_address}).")